In [55]:
%reload_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import datetime
import sys
from collections import Counter

import os
import re
from robobrowser import RoboBrowser
import re
import pandas as pd 
import sqlite3
from contextlib import closing
from preprocessing import make_lines,make_stopword,clean_txt,count_matubi,count_to_pd,count_morpheme

In [56]:
dbname='review_of_bakawall.db'
min_len=20
conn=sqlite3.connect(dbname)
c=conn.cursor()



In [57]:
def main(mor,M):
    count_df=pd.DataFrame()
    for i in range(1,6):
        word_counter = Counter()
        select_sql=f'''
        select *from reviews where rate={i}
        and len >{min_len}'''
        txts=[]
        for row in c.execute(select_sql):
            txt=clean_txt(row[4])
            txts.extend(txt)
        if M==0:
            cnt1=count_morpheme(txts,word_counter,mor)
        else:
            cnt1=count_matubi(txts,word_counter)
            
        count_df=count_to_pd(count_df,str(i),cnt1)
    return(count_df)

In [59]:
df_original=main('hoge',1)


In [78]:
df=df_original.copy()
df=df.fillna(0)

In [79]:
df['sum']=df.sum(numeric_only=True,axis=1)
df['low_sum']=df[['1','2']].sum(numeric_only=True,axis=1)
df['high_sum']=df[['4','5']].sum(numeric_only=True,axis=1)
df['high_low_ratio']=df['low_sum']/df['high_sum']

In [80]:
df=df.sort_values('high_low_ratio',ascending=False)



In [82]:
import codecs
# codecs.ignore_errors(UnicodeEncodeError)
codecs.register_error('\ufe0e', lambda e: ('', e.end))
df.to_csv('baka_wall_matubi.csv',encoding='cp932')


UnicodeEncodeError: 'cp932' codec can't encode character '\ufe0e' in position 0: illegal multibyte sequence